<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/03_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Some SQL with BigQuery

The first bit below comes directly from Google, you'll need to do each of those to be successful in getting this document to work.  

## Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Now that I am authenticated, I can start to play around in the dataset.  I am going to look at the liquor sales data from Iowa and try to find the most and least sales by city.  I do have a project called `pic-math` in my BigQuery interface.  So you'll need to make one but keep the name simple but identifiable!

## Why do we use SQL

Below you'll see a basic SQL call.  This illustrates why excel is not useful, 22 million rows is about 21.5 million more than excel can handle!  Essentially SQL will do the data manipulations on the database server side instead of on you machine (or in the cloud with colab)

In [ ]:
%%bigquery --project pic-math
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.iowa_liquor_sales.sales`

,total_rows
0,22494544


We see that is a lot of rows.  We really don't want to try to store that into memory!  Let's have a peak at the data.

The only two required features of an SQL call are `SELECT` and `FROM`.  `SELECT` picks the columns you want by name in the data table.  `FROM` picks the table you want to look at.  Both can be shorted in the call and sometimes it is nesseccary to all the table name with the column.  Below I do the same thing in two different ways.  Do you see a difference in the output?

In [ ]:
%%bigquery --project pic-math
SELECT 
  AVG(sale_dollars)
FROM `bigquery-public-data.iowa_liquor_sales.sales`

,f0_
0,138.881749


In [ ]:
%%bigquery --project pic-math
SELECT AVG(table.sale_dollars) as average_sale_dollars
FROM `bigquery-public-data.iowa_liquor_sales.sales` as table

,average_sale_dollars
0,138.881749


See any difference?  You should be asking yourself why it would be adventageous to name your tables.  Well, we will see shortly that joining the tables (remember relational database?) is going to be an important task!  Sometimes some info we want will be in one table and some of it in another.

Here is a command that will allow you to peak at the whole table (like head!)  The `*` gives you all the columns and the `LIMIT` gives only how many you specify.  There is not preferred order in SQL.

In [ ]:
%%bigquery --project pic-math
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-20825400070,2019-07-24,2555,Hy-Vee Food Store / Keokuk,3111 Main,Keokuk,52632.0,POINT (-91.403752 40.41538),56,LEE,None,None,55,SAZERAC NORTH AMERICA,84197,99 Pineapple Mini,10,600,6.00,9.00,10,90.00,6.00,1.58
1,INV-33554300013,2021-01-18,4493,Casey's General Store #2850 / Cedar Rapids,5655 C St SW,Cedar Rapids,52404.0,POINT (-91.637306 41.922505),57,LINN,1081100.0,Coffee Liqueurs,370.0,PERNOD RICARD USA,67524,Kahlua Coffee,24,375,6.49,9.74,5,48.70,1.87,0.49
2,INV-17816400003,2019-02-27,4069,Fareway Stores #648 / Ottumwa,1325 Albia Rd,Ottumwa,52501,POINT (-92.440213 41.00934),90,WAPELLO,1901200.0,Special Order Items,259.0,Heaven Hill Brands,917914,HA Elijah Craig Cask Barrel Proof,3,750,29.92,44.88,9,403.92,6.75,1.78
3,INV-23344500010,2019-11-20,4324,Dayton Community Grocery,22 North Main,Dayton,50530.0,POINT (-94.068439 42.26168),94,WEBSTER,1081100.0,Coffee Liqueurs,370.0,PERNOD RICARD USA,67527,Kahlua Coffee,12,1000,15.15,22.73,5,113.65,5.00,1.32
4,S09220000195,2012-11-29,2560,Hy-Vee Food Store / Marion,3600 BUSINESS HWY 151 EAST,MARION,52302,None,57,Linn,1081350.0,SPEARMINT SCHNAPPS,434,Luxco-St Louis,82187,Arrow Spearmint Schnapps,12,1000,4.75,7.13,4,28.52,4.00,1.06


Not sure how much we might use this but if I wanted the data into a `pandas` dataframe, I just add a name for it after the bigquery project name.

In [ ]:
%%bigquery --project pic-math df
SELECT 
  city, 
  store_name,
  SUM(sale_dollars) as total_sales
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE city is not null
GROUP BY city, store_name

In [ ]:
df

,city,store_name,total_sales
0,Keokuk,Hy-Vee Food Store / Keokuk,4226546.86
1,Cedar Rapids,Casey's General Store #2850 / Cedar Rapids,277130.53
2,Ottumwa,Fareway Stores #648 / Ottumwa,1853189.35
3,Dayton,Dayton Community Grocery,312445.18
4,MARION,Hy-Vee Food Store / Marion,4305523.82
...,...,...,...
4144,Cedar Falls,Happy's Wine & Spirits Wholesale,1550.99
4145,Bettendorf,"Cats Eye Distillery, LLC",693.12
4146,Colorado Springs,Dash Events LLC,31.14
4147,Keosauqua,Honey Creek Distillery,235.68


In [ ]:
import pandas as pd

groupeddf = df.groupby('city')

In [ ]:
maxdf = groupeddf.max()

In [ ]:
maxdf

,store_name,total_sales
city,,
ACKLEY,KUM & GO #513 / ACKLEY,132551.63
ADAIR,KUM & GO #76 / ADAIR,148812.43
ADEL,Shugar's Super Valu / Adel,982611.78
AFTON,Casey's General Store #2626 / Afton,137832.15
AKRON,Casey's General Store #2425 / Akron,134999.66
...,...,...
Woodbine,Foodland Super Markets / Woodbine,385599.54
Woodward,Casey's General Store #2551 / Woodward,186083.15
Wyoming,Casey's General Store #1126 / Wyoming,35499.38


In [ ]:
mindf = groupeddf.min()

mindf

,store_name,total_sales
city,,
ACKLEY,Ackley Super Foods,108359.95
ADAIR,Casey's General Store #2521 / Adair,40784.57
ADEL,Casey's General Store #1680 / Adel,19733.51
AFTON,Casey's General Store #2626 / Afton,137832.15
AKRON,Casey's General Store #2425 / Akron,134999.66
...,...,...
Woodbine,Casey's General Store #2665 / Woodbine,6439.33
Woodward,Casey's General Store #2551 / Woodwa,9095.21
Wyoming,Casey's General Store #1126 / Wyoming,35499.38


I am clearly just showing off now.  I have left more along this line at the bottom but let's get your assignment up!

Assignement for today

1. Start a notebook getting BigQuery to work.  Feel free to use the authentication atop.
2. Navigate to the dataset 'austin_bikeshare.bikeshare_trips'
3. Compute how many entries are in the dataset
4. Compute the longest trip from 'duration_minutes'
5. Compute the average time for a trip

## More on Liquors not Needed today

I notice a few things attempting this.  While I think I have a solution, it is clearly not the best.  Zwingle and ZWINGLE are probably the same town and SNK may just be the only store but the fact that it appears four times in my lists is disappointing!

In [ ]:
maxdf.sort_values('total_sales',ascending=False)

,store_name,total_sales
city,,
Des Moines,Walgreens #07968 / Des Moines,5.785247e+07
DES MOINES,Walgreens #07968 / Des Moines,4.014780e+07
Iowa City,goPuff / Iowa City,2.576451e+07
Cedar Rapids,"Williams Boulevard Service, Inc.",1.883548e+07
Mount Vernon,Wilkie Liquors,1.794630e+07
...,...,...
RUNNELLS,Casey's General Store #2683 / Runn,5.495630e+03
GILBERTVILLE,Casey's General Store #2900 / Gilber,4.947320e+03
Malcom,Malcom Gas & Food / Malcom,4.493810e+03


I want to try the extra challenge **and** do it all in SQL.  I'll try to find which store had the most sales by date!

In [ ]:
%%bigquery --project pic-math

WITH bestday as (
SELECT 
  date, 
  store_name,
  city,
  SUM(sale_dollars) as total_sales,
  RANK() over (PARTITION BY date ORDER BY SUM(sale_dollars) desc) as top_sales_rank
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE city is not null
GROUP BY date, store_name, city
)

SELECT
  date,
  store_name,
  city,
  total_sales
FROM bestday
WHERE top_sales_rank = 1
ORDER BY date

,date,store_name,city,total_sales
0,2012-01-03,Sam's Club 8238 / Davenport,DAVENPORT,51350.70
1,2012-01-04,Hy-Vee Food Store / Sheldon,SHELDON,73296.69
2,2012-01-05,Hy-Vee #3 / BDI / Des Moines,DES MOINES,74085.23
3,2012-01-09,"Central City Liquor, Inc.",DES MOINES,60435.09
4,2012-01-10,Sam's Club 8238 / Davenport,DAVENPORT,33423.60
...,...,...,...,...
2443,2021-10-26,Costco Wholesale #788 / WDM,West Des Moines,158640.30
2444,2021-10-27,I-80 Liquor / Council Bluffs,Council Bluffs,89184.13
2445,2021-10-28,Central City 2,Des Moines,106029.76
2446,2021-10-29,Foundry Distilling Company,West Des Moines,55950.00
